In [5]:
import re
import os
import sys
from dotenv import load_dotenv
import boto3
import pandas as pd

from datetime import datetime

dir_resource_minio = os.path.join(os.getcwd(), '..', 'src', 'resources')
sys.path.append(dir_resource_minio)

from boto3_manager import PandasBucket


load_dotenv(os.path.join(os.getcwd(), '..', '.env'))

client = boto3.client(
    's3',
    endpoint_url='http://localhost:9000',  # Correct API port
    aws_access_key_id=os.getenv('ACCESS_KEY'),
    aws_secret_access_key=os.getenv('SECRET_KEY'),
    region_name='us-east-1'
)

In [6]:
bucket_resientevil = PandasBucket(client=client, name='resident-evil')

In [7]:
data = bucket_resientevil.read_json_from_s3(name='bronze/person_characters.json')

In [8]:
df = pd.DataFrame(data)

In [ ]:
def extract_year(value) -> str:
    if isinstance(value, str):
        year = value.strip()[:4]
        if year.isdigit():
            year = int(year)
            current_year = datetime.now().year
            if 1900 <= year <= current_year:
                return str(year)
    return 'Desconhecido'

def extract_type_sanguine(value: str) -> str:
    if isinstance(value, str):
        value = value.strip().lower()
        map = {
            '\xa0desconhecido.': 'Desconhecido',
            'desconhecido.': 'Desconhecido',
            'desconhecido': 'Desconhecido',
            'desconhecido': 'Desconhecido'
        }
        return map.get(value, value.upper())
    return 'Desconhecido'

def extract_height(value) -> str:
    # Verifica se o valor é uma string
    if isinstance(value, str):
        # Busca o primeiro padrão que corresponde a altura (ex: 1,75m ou 1,75cm)
        match = re.search(r'\b\d+,\d+(?:m|cm)\b', value)
        if match:
            return match.group(0)
    return "Desconhecido"

def extract_weight(value) -> str:
    # Verifica se o valor é uma string
    if isinstance(value, str):
        # Busca o primeiro padrão que corresponde ao peso (ex: 84,5kg ou 84kg)
        match = re.search(r'\b\d+(?:,\d+)?kg\b', value)
        if match:
            return match.group(0)
    return "Desconhecido"

In [ ]:
df['ano_de_nascimento'] = df['Ano de nascimento'].fillna(df['de nascimento']).apply(extract_year)
df['tipo_sanguineo'] = df['Tipo sanguíneo'].apply(extract_type_sanguine)
df['altura'] = df['Altura'].apply(extract_height)
df['peso'] = df['Peso'].apply(extract_weight)
df['IngestionDate'] = datetime.now().strftime('%Y-%m-%d')
df['IngestionTime'] = datetime.now().strftime('%H:%M:%S')
df['Source'] = 'DataResidentEvil'

In [132]:
df = df.drop(columns={'Ano de nascimento', 'de nascimento', 'Tipo sanguíneo', 'Altura', 'Peso'})

In [142]:
df

,aparicoes,link,name,ano_de_nascimento,tipo_sanguineo,altura,peso,IngestionDate,IngestionTime,Source
0,"[Biohazard / Resident Evil 2 (1998), Biohazard...",https://www.residentevildatabase.com/ada-wong/,ada-wong,1974,AB,Desconhecido,Desconhecido,2024-11-10,11:59:22,DataResidentEvil
1,[Biohazard / Resident Evil 6 (2012)],https://www.residentevildatabase.com/adam-benf...,adam-benford,Desconhecido,Desconhecido,Desconhecido,Desconhecido,2024-11-10,11:59:22,DataResidentEvil
2,"[Biohazard / Resident Evil 7 (2017), Biohazard...",https://www.residentevildatabase.com/alan-doug...,alan-douglas,Desconhecido,Desconhecido,Desconhecido,Desconhecido,2024-11-10,11:59:22,DataResidentEvil
3,"[Biohazard / Resident Evil Clássico (1996), Bi...",https://www.residentevildatabase.com/albert-we...,albert-wesker,1960,O,"1,83cm","84,5kg",2024-11-10,11:59:22,DataResidentEvil
4,[Biohazard / Resident Evil Village (2021)],https://www.residentevildatabase.com/alcina-di...,alcina-dimitrescu,Desconhecido,Desconhecido,"2,90m",Desconhecido,2024-11-10,11:59:22,DataResidentEvil
...,...,...,...,...,...,...,...,...,...,...
158,"[Biohazard / Resident Evil 0 (2003), Biohazard...",https://www.residentevildatabase.com/william-b...,william-birkin,1962,O,"1,78cm","66,7kg",2024-11-10,11:59:22,DataResidentEvil
159,[Biohazard / Resident Evil: Operation Raccoon ...,https://www.residentevildatabase.com/willow-ca...,willow-caroline-floyd,Desconhecido,Desconhecido,Desconhecido,Desconhecido,2024-11-10,11:59:22,DataResidentEvil
160,[BIOHAZARD / RESIDENT EVIL: Infinite Darkness ...,https://www.residentevildatabase.com/wilson-se...,wilson-secretario-seguranca,Desconhecido,Desconhecido,Desconhecido,Desconhecido,2024-11-10,11:59:22,DataResidentEvil
161,"[Biohazard / Resident Evil Outbreak (2003), Bi...",https://www.residentevildatabase.com/yoko-suzuki/,yoko-suzuki,1978,Desconhecido,"1,63m",58kg,2024-11-10,11:59:22,DataResidentEvil
